In [1]:
import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree 
from sklearn import metrics
import matplotlib.pyplot as plt
from apriori_python import apriori

from csv import reader
from collections import defaultdict
from itertools import chain, combinations
from optparse import OptionParser
from apriori_python.utils import *

C:\Users\diego\AppData\Local\Temp\ipykernel_27484\2226442294.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
DB = {'servername': 'LAPTOP-3S422SS6\\SQLEXPRESS', 'database': 'DAD'}

conn_str = (
    'DRIVER={SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'Trusted_Connection=yes;'
)

conn = pyodbc.connect(conn_str)

query = "SELECT * FROM ORDERR"
order = pd.read_sql_query(query, conn)

query = "SELECT * FROM PRODUCT"
product = pd.read_sql_query(query, conn)

conn.close()

In [24]:
df = pd.merge(order,product,left_on='PRODUCT_number',how='left',right_on='PRODUCT_id')
df = df[['ORDER_detail_id','PRODUCT_name']]
df_grouped = df.groupby('ORDER_detail_id')['PRODUCT_name'].agg(list).reset_index()
df_grouped['PRODUCT_name'] = df_grouped['PRODUCT_name'].apply(lambda x: list(set(x)))
itemSetList = df_grouped['PRODUCT_name'].tolist()


In [4]:
def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet

def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet

def apriori(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result, global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

In [29]:
minSup = 0.02  # 2% steun
minConf = 0.66 # 

result = apriori(itemSetList, minSup, minConf)

In [44]:
list = pd.DataFrame(result, columns=[1,2,3,4,5], index=[1,2])
list

,1,2,3,4,5
1,"{(Mountain Man Deluxe), (TrailChef Kitchen Kit...","{(Firefly Extreme, TrailChef Water Bag), (Husk...",NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
